In [1]:
import os, re, html
import numpy as np
import pandas as pd
import folium
from folium import Element, Popup

# -------------------- CONFIG --------------------
CSV_PATH = "Miami dev site deals - Sheet1.csv"

PROPERTY_COL = "Name (site/project)"
COORD_COL    = "Google coordinates"
CAPTION_COL  = "Caption"

LAT_COL = "__lat"
LON_COL = "__lon"
STATUS_COL = "__geocode_status"

MAPBOX_STYLE = "mapbox/streets-v11"

DISPLAY_COLUMNS = [
    "Name (site/project)",
    "Caption",
    "Buyer",
    "Seller",
    "Price",
    "Price per acre",
    "Month + Year Closed",
    "Link",
]
# ------------------------------------------------

# ---- Load Mapbox token (try %store first, then env) ----
try:
    get_ipython().run_line_magic("store", "-r map_box_api_key")
except Exception:
    pass  # not in IPython or no stored variable

if 'map_box_api_key' not in globals() or not map_box_api_key:
    map_box_api_key = (
        os.getenv("MAPBOX_TOKEN")
        or os.getenv("MAPBOX_ACCESS_TOKEN")
        or os.getenv("MAPBOX_API_KEY", "")
    )

if not map_box_api_key:
    raise ValueError(
        "No Mapbox token found. Set %store map_box_api_key or env var "
        "MAPBOX_TOKEN / MAPBOX_ACCESS_TOKEN / MAPBOX_API_KEY."
    )

# ---- Load CSV ----
df = pd.read_csv(CSV_PATH)
df.columns = [c.strip() for c in df.columns]

missing_cols = [c for c in [PROPERTY_COL, COORD_COL, CAPTION_COL] if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing expected columns {missing_cols}. Got: {df.columns.tolist()}")

# Clean property strings
df[PROPERTY_COL] = df[PROPERTY_COL].astype(str).str.strip()

# Ensure lat/lon/status columns exist (robust if your file sometimes lacks them)
if LAT_COL not in df.columns:
    df[LAT_COL] = np.nan
if LON_COL not in df.columns:
    df[LON_COL] = np.nan
if STATUS_COL not in df.columns:
    df[STATUS_COL] = ""

# Coerce existing lat/lon to numeric
df[LAT_COL] = pd.to_numeric(df[LAT_COL], errors="coerce")
df[LON_COL] = pd.to_numeric(df[LON_COL], errors="coerce")

# ---- Parse coordinates from "Google coordinates" (only if lat/lon missing) ----
# Handles:
#   "26.7145,-80.0523"
#   "26.7145, -80.0523"
#   "(26.7145, -80.0523)"
coord_re = re.compile(r"(-?\d+(?:\.\d+)?)\s*[, ]\s*(-?\d+(?:\.\d+)?)")
new_statuses = []

for i, val in df[COORD_COL].items():
    already_has = pd.notna(df.at[i, LAT_COL]) and pd.notna(df.at[i, LON_COL])
    if already_has:
        new_statuses.append("CSV_EXISTING")
        continue

    if pd.isna(val):
        new_statuses.append("CSV_EMPTY")
        continue

    s = str(val).strip()
    m = coord_re.search(s)
    if not m:
        new_statuses.append("CSV_INVALID")
        continue

    lat = float(m.group(1))
    lon = float(m.group(2))

    df.at[i, LAT_COL] = lat
    df.at[i, LON_COL] = lon
    new_statuses.append("CSV_PARSED")

df[STATUS_COL] = new_statuses

mapped = df.dropna(subset=[LAT_COL, LON_COL]).copy()
print(f"Rows with coordinates: {len(mapped)} / {len(df)}")
if mapped.empty:
    raise RuntimeError("No coordinates produced from __lat/__lon or Google coordinates — check formats.")

# ---- HTML helpers ----
def _clean(v):
    if pd.isna(v):
        return None
    s = str(v).strip()
    return s if s and s.lower() not in {"nan", "none"} else None

def make_popup_html_lines(row) -> str:
    blocks = []
    for col in DISPLAY_COLUMNS:
        if col not in row.index:
            continue

        val = _clean(row.get(col))
        if val is None:
            continue

        if col == "Link" and isinstance(val, str) and val.lower().startswith(("http://", "https://")):
            rendered = f'<a href="{html.escape(val)}" target="_blank" rel="noopener">Open link</a>'
        else:
            rendered = html.escape(str(val))

        blocks.append(
            f"<div style='margin:0 0 6px 0;'>"
            f"  <div style='font-weight:700; margin:0;'>{html.escape(col)}</div>"
            f"  <div style='white-space:pre-wrap; margin:0;'>{rendered}</div>"
            f"</div>"
        )

    if not blocks:
        return "<i>No details</i>"

    return "<div>" + "".join(blocks) + "</div>"


# ---- Build map ----
center = [float(mapped[LAT_COL].mean()), float(mapped[LON_COL].mean())]
m = folium.Map(
    location=center,
    zoom_start=12,
    zoom_control=False,
    control_scale=True,
    tiles=(
        f"https://api.mapbox.com/styles/v1/{MAPBOX_STYLE}/tiles/256/{{z}}/{{x}}/{{y}}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="© Mapbox © OpenStreetMap",
    name="Base",
)

# ---- Global popup sizing + scroll behavior ----
popup_css = """
<style>
.leaflet-popup-content-wrapper {
  max-width: 900px;
}
.leaflet-popup-content {
  width: 400px !important;
  max-height: 65vh;
  overflow-y: auto;
  overflow-x: hidden;

  /* tighter typography */
  font-family: Arial, sans-serif;
  font-size: 12.5px;
  line-height: 1.25;
}
</style>
"""
m.get_root().html.add_child(Element(popup_css))


# Optional: title
title_html = """
<h3 align="center" style="font-size:20px; margin-top:10px; margin-bottom:5px;">
  <b>Miami dev site deals</b>
</h3>
"""
m.get_root().html.add_child(Element(title_html))

intro_panel = """
<style>
#intro-panel{
  position: absolute;
  top: 70px;
  left: 10px;
  width: 380px;
  max-width: 42vw;
  max-height: calc(100vh - 100px);
  overflow-y: auto;

  background: rgba(255,255,255,0.97);
  padding: 12px 14px;
  border-radius: 10px;
  box-shadow: 0 0 10px rgba(0,0,0,0.25);
  z-index: 9999;

  font-family: Arial, sans-serif;
  font-size: 13px;
  line-height: 1.35;
}
#intro-panel h4{
  margin: 0 0 8px 0;
  font-size: 15px;
}
#intro-panel p{
  margin: 0 0 8px 0;
}
</style>

<div id="intro-panel">
  <h4>About these deals</h4>
  <p>Oak Row Equities and OKO Group, led by Vlad Doronin, closed on the $520 million purchase of a 4.25-acre development site in Miami's Brickell this month. It marked the biggest known land deal in South Florida and likely the state. Still, it wasn't the top deal on a price per acre basis.</p>
  <p>The Real Deal compiled some of the other notable development site deals in South Florida over the years. Most were for property in downtown Miami or Brickell. Here's who else paid how much for prime land:</p>
</div>
"""
m.get_root().html.add_child(Element(intro_panel))


# ---- Add markers (click -> popup) ----
for _, r in mapped.iterrows():
    folium.CircleMarker(
        location=(float(r[LAT_COL]), float(r[LON_COL])),
        radius=7,
        color="red",
        weight=2,
        fill=True,
        fill_color="red",
        popup=Popup(make_popup_html_lines(r), max_width=900),
    ).add_to(m)

# ---- Save ----
m.save("index.html")
print("✅ Saved map to index.html")

m


Rows with coordinates: 6 / 6
✅ Saved map to index.html


In [2]:
# <div id="info-panel">
#   <h4>About this map</h4>
#   <p>Across South Florida, at least 48 multifamily properties qualified this year for a property tax abatement through the Live Local Act. Most –– if not all –– weren’t originally developed under the state’s affordable and workforce housing law. Instead, landlords sought the tax breaks after the fact.</p>
#   <p>Live Local gives landlords a full property tax exemption for units rented to households earning up to 80 percent of the area median income, and a 75 percent abatement for units rented to households earning from 80 percent to 120 percent of the AMI.</p>
#   <p>Some landlords dropped their asking rents in order to qualify for the Live Local tax breaks –– which also allowed them to speed up leasing amid South Florida’s market-rate apartment glut and high demand for rentals at workforce rents. Others had already planned to offer workforce-level rents, meaning they didn’t need to adjust pricing to receive the abatements.</p>
# </div>

In [3]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDevSiteDeals_12_29_2025
